### Data format corrections

In [5]:
import numpy as np
import pandas as pd

import datetime
from collections import defaultdict
import pybgpstream

In [6]:
# import modin.pandas as pd
# import ray
# ray.init(runtime_env={'env_vars': {'__MODIN_AUTOIMPORT_PANDAS__': '1'}})

In [7]:
dfr = pd.read_hdf("../data/01_raw/googletest-small.h5", key='raw')

In [8]:
dfr

,type,time,peer_asn,peer_ip,prefix,as_path
0,A,1.396368e+09,25152,202.249.2.185,177.107.11.0/24,25152 2914 1239 16735 53059 262730
1,A,1.396368e+09,25152,202.249.2.185,190.211.244.0/24,25152 6939 12956 22927 52295
2,A,1.396368e+09,25152,202.249.2.185,190.211.245.0/24,25152 6939 12956 22927 52295
3,A,1.396368e+09,25152,202.249.2.185,177.107.11.0/24,25152 6939 16735 53059 262730
4,A,1.396368e+09,25152,202.249.2.185,177.107.11.0/24,25152 6939 3549 16735 53059 262730
...,...,...,...,...,...,...
20704,A,1.396372e+09,25152,202.249.2.185,94.139.170.0/23,25152 6939 8529 12880 25124 57218 57218 57218
20705,A,1.396372e+09,25152,202.249.2.185,188.253.120.0/22,25152 6939 8529 48159 58127
20706,A,1.396372e+09,25152,202.249.2.185,5.233.125.0/24,25152 6939 8529 48159
20707,A,1.396372e+09,25152,202.249.2.185,66.210.60.0/24,25152 6939 22773 20450


In [9]:
dfr = dfr.drop(columns=["peer_ip"])

In [10]:
# r.time
# datetime.datetime.utcfromtimestamp(r.time).strftime('%Y-%m-%d %H:%M:%S.%f')
dfr.time = pd.to_datetime(dfr["time"]*10e8, unit='ns')

In [11]:
dfr["type"] = dfr.apply(lambda x: True if x.type == "A" else False, axis=1)

In [12]:
dfr["peer_asn"] = dfr["peer_asn"].astype("category")

In [13]:
dfr["prefix_ip"], dfr["prefix_nm"] = zip(*dfr["prefix"].apply(lambda x: x.split('/')))

In [14]:
# dfr = dfr.astype({"peer_asn": np.uint32,"prefix_nm":np.uint8 })
dfr = dfr.astype({"prefix_nm":np.uint8 })

In [15]:
# np.array(dfr["peer_ip"].apply(lambda x: x.split('.')), dtype=np.uint8)
# print(set(zip(*dfr["peer_ip"].apply(lambda x: x.split('.')))))

In [16]:
# dftmp = pd.DataFrame(dfr["prefix_ip"].apply(lambda x: np.array(x.split('.'),dtype=np.uint8 )).tolist())
# dfr["peer_ip_a"], dfr["peer_ip_b"], dfr["peer_ip_c"], dfr["peer_ip_d"],  = zip(*dfr["peer_ip"].apply(lambda x: np.array(x.split('.'))))

#disabled temp
# dfr["prefix_ip_a"], dfr["prefix_ip_b"], dfr["prefix_ip_c"], dfr["prefix_ip_d"]  = zip(*dfr["prefix_ip"].apply(lambda x: np.array(x.split('.'))))
# dfr["prefix_ip_a"] = dfr["prefix_ip_a"].astype(np.uint8)
# dfr["prefix_ip_b"] = dfr["prefix_ip_b"].astype(np.uint8)
# dfr["prefix_ip_c"] = dfr["prefix_ip_c"].astype(np.uint8)
# dfr["prefix_ip_d"] = dfr["prefix_ip_d"].astype(np.uint8)

In [17]:
# def ipstr2ipint(ipstr):
#     parts = ipstr.split('.')
#     return (int(parts[0]) << 24) + (int(parts[1]) << 16) + (int(parts[2]) << 8) + int(parts[3])

In [18]:
# from cyberpandas import to_ipaddress
# # dfr["peer_ip"] = to_ipaddress(dfr["peer_ip"])
# dfr["prefix_ip"] = to_ipaddress(dfr["prefix_ip"])
# dfr["prefix_ip_int"] = dfr["prefix_ip"].apply(lambda x: int(x))

In [19]:
dfr["prefix_ip_cat"] = dfr["prefix"].astype("category")

In [20]:
# dfr = dfr.drop(columns=["prefix"])


In [21]:
dfr.loc[dfr["prefix_nm"] == 25]

,type,time,peer_asn,prefix,as_path,prefix_ip,prefix_nm,prefix_ip_cat
949,True,2014-04-01 16:02:47,25152,91.212.118.0/25,25152 2914 9002 5563 49208 49208,91.212.118.0,25,91.212.118.0/25


In [22]:
# dfr[["type", 	"time" ,	"peer_asn" ,	"as_path" 	, 	"prefix_nm" ,	"prefix_ip_a" ,	"prefix_ip_b" ,	"prefix_ip_c", 	"prefix_ip_d", 	"prefix_ip_cat"]]
dfr

,type,time,peer_asn,prefix,as_path,prefix_ip,prefix_nm,prefix_ip_cat
0,True,2014-04-01 16:00:02,25152,177.107.11.0/24,25152 2914 1239 16735 53059 262730,177.107.11.0,24,177.107.11.0/24
1,True,2014-04-01 16:00:02,25152,190.211.244.0/24,25152 6939 12956 22927 52295,190.211.244.0,24,190.211.244.0/24
2,True,2014-04-01 16:00:02,25152,190.211.245.0/24,25152 6939 12956 22927 52295,190.211.245.0,24,190.211.245.0/24
3,True,2014-04-01 16:00:02,25152,177.107.11.0/24,25152 6939 16735 53059 262730,177.107.11.0,24,177.107.11.0/24
4,True,2014-04-01 16:00:02,25152,177.107.11.0/24,25152 6939 3549 16735 53059 262730,177.107.11.0,24,177.107.11.0/24
...,...,...,...,...,...,...,...,...
20704,True,2014-04-01 16:59:55,25152,94.139.170.0/23,25152 6939 8529 12880 25124 57218 57218 57218,94.139.170.0,23,94.139.170.0/23
20705,True,2014-04-01 16:59:55,25152,188.253.120.0/22,25152 6939 8529 48159 58127,188.253.120.0,22,188.253.120.0/22
20706,True,2014-04-01 16:59:56,25152,5.233.125.0/24,25152 6939 8529 48159,5.233.125.0,24,5.233.125.0/24
20707,True,2014-04-01 16:59:57,25152,66.210.60.0/24,25152 6939 22773 20450,66.210.60.0,24,66.210.60.0/24


## Path

In [23]:
# t = "25152 6939 23520 26617 26617 26617 {65460,65470}"
# import re
# #re.split(' |,|{|}',t)

# path_pattern = re.compile('(\d+)+')
# path_str = path_pattern.findall(t)
# path_np = np.array(path_str, dtype=np.uint32)

# path_np

In [24]:
import re
path_pattern = re.compile('(\d+)+')
# asp='1836 174 7922 33287 {21669}'
# 25152 6939 23520 26617 26617 26617 {65460,65470}
# asp.replace('{','').replace('}','').split(' ')
def process_as_path(asp):
    if type(asp) == np.ndarray:
        return asp, 0
    else:
        # print(asp)
        # npasp = np.array(asp.replace('{','').replace('}','').split(' '), dtype=np.uint32)
        # npasp = np.array(asp.replace('{','').replace('}','').split(' '), dtype=np.uint32)
        path_str = path_pattern.findall(asp)
        npasp = np.array(path_str, dtype=np.uint32)
        return npasp, len(npasp)

In [25]:
# dfr[dfr["type"] == True]["as_path"].apply(lambda x: x.replace('{','').replace('}','').split(' '))
# dfr["as_path"] = dfr[dfr["type"] == True]["as_path"].apply(lambda x: np.array(x.replace('{','').replace('}','').split(' '), dtype=np.uint32))
# dfr["as_path_len"] = dfr[dfr["type"] == True]["as_path"].apply(lambda x: len(x))

In [26]:
dfr["as_path"], dfr["as_path_len"] = zip(*dfr["as_path"].apply(process_as_path))

In [27]:
# dfr["as_path"], dfr["as_path_len"] = zip(*dfr[dfr["type"] == True]["as_path"].apply(process_as_path))

In [28]:
dfr

,type,time,peer_asn,prefix,as_path,prefix_ip,prefix_nm,prefix_ip_cat,as_path_len
0,True,2014-04-01 16:00:02,25152,177.107.11.0/24,"[25152, 2914, 1239, 16735, 53059, 262730]",177.107.11.0,24,177.107.11.0/24,6
1,True,2014-04-01 16:00:02,25152,190.211.244.0/24,"[25152, 6939, 12956, 22927, 52295]",190.211.244.0,24,190.211.244.0/24,5
2,True,2014-04-01 16:00:02,25152,190.211.245.0/24,"[25152, 6939, 12956, 22927, 52295]",190.211.245.0,24,190.211.245.0/24,5
3,True,2014-04-01 16:00:02,25152,177.107.11.0/24,"[25152, 6939, 16735, 53059, 262730]",177.107.11.0,24,177.107.11.0/24,5
4,True,2014-04-01 16:00:02,25152,177.107.11.0/24,"[25152, 6939, 3549, 16735, 53059, 262730]",177.107.11.0,24,177.107.11.0/24,6
...,...,...,...,...,...,...,...,...,...
20704,True,2014-04-01 16:59:55,25152,94.139.170.0/23,"[25152, 6939, 8529, 12880, 25124, 57218, 57218...",94.139.170.0,23,94.139.170.0/23,8
20705,True,2014-04-01 16:59:55,25152,188.253.120.0/22,"[25152, 6939, 8529, 48159, 58127]",188.253.120.0,22,188.253.120.0/22,5
20706,True,2014-04-01 16:59:56,25152,5.233.125.0/24,"[25152, 6939, 8529, 48159]",5.233.125.0,24,5.233.125.0/24,4
20707,True,2014-04-01 16:59:57,25152,66.210.60.0/24,"[25152, 6939, 22773, 20450]",66.210.60.0,24,66.210.60.0/24,4


In [29]:
# dfr["as_path"].apply(lambda x: type(x)).value_counts()

In [30]:
# dfr.to_hdf("googletest-raw.h5", key='preprocessed', orient="table")
dfr.to_json("../data/02_intermediate/googletest-test.json", orient="table")
# dfr.to_hdf("googletest-raw.h5", key='path', format="table")

## Prefix one-hot encoding 

In [31]:
dfr.memory_usage()
dfr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20709 entries, 0 to 20708
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   type           20709 non-null  bool          
 1   time           20709 non-null  datetime64[ns]
 2   peer_asn       20709 non-null  category      
 3   prefix         20709 non-null  object        
 4   as_path        20709 non-null  object        
 5   prefix_ip      20709 non-null  object        
 6   prefix_nm      20709 non-null  uint8         
 7   prefix_ip_cat  20709 non-null  category      
 8   as_path_len    20709 non-null  int64         
dtypes: bool(1), category(2), datetime64[ns](1), int64(1), object(3), uint8(1)
memory usage: 1.7+ MB


In [32]:
# from sklearn import preprocessing
# lb = preprocessing.LabelBinarizer()
# y = dfr["prefix_ip"]
# ylab = lb.fit(y)
# df = pd.DataFrame()

In [33]:
# from sklearn import preprocessing
# lb = preprocessing.LabelBinarizer()
# dfr = dfr.join(pd.DataFrame(lb.fit_transform(dfr["prefix_ip"]), columns=lb.classes_, index=dfr.index))

In [34]:
# oneliner:
# dfr = dfr.join(pd.DataFrame(lb.fit_transform(dfr["prefix_ip"]), columns=lb.classes_, index=dfr.index))

In [48]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
# nmlab = lb.fit(dfr["prefix_nm"])
nmlab = lb.fit(np.arange(0, 33))

nmpredf = lb.transform(dfr["prefix_nm"])

In [50]:
nmpredfspar = pd.DataFrame(nmpredf, columns=lb.classes_, index=dfr.index, dtype="Sparse[int8]")
dfr = dfr.join(nmpredfspar)

In [35]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
ylab = lb.fit(dfr["prefix_ip"])
predf = lb.transform(dfr["prefix_ip"])


In [37]:
predfspar = pd.DataFrame(predf, columns=lb.classes_, index=dfr.index, dtype="Sparse[int8]")

In [39]:
dfr = dfr.join(predfspar)

In [51]:
dfr

,type,time,peer_asn,prefix,as_path,prefix_ip,prefix_nm,prefix_ip_cat,as_path_len,1.186.68.0,...,23,24,25,26,27,28,29,30,31,32
0,True,2014-04-01 16:00:02,25152,177.107.11.0/24,"[25152, 2914, 1239, 16735, 53059, 262730]",177.107.11.0,24,177.107.11.0/24,6,0,...,0,1,0,0,0,0,0,0,0,0
1,True,2014-04-01 16:00:02,25152,190.211.244.0/24,"[25152, 6939, 12956, 22927, 52295]",190.211.244.0,24,190.211.244.0/24,5,0,...,0,1,0,0,0,0,0,0,0,0
2,True,2014-04-01 16:00:02,25152,190.211.245.0/24,"[25152, 6939, 12956, 22927, 52295]",190.211.245.0,24,190.211.245.0/24,5,0,...,0,1,0,0,0,0,0,0,0,0
3,True,2014-04-01 16:00:02,25152,177.107.11.0/24,"[25152, 6939, 16735, 53059, 262730]",177.107.11.0,24,177.107.11.0/24,5,0,...,0,1,0,0,0,0,0,0,0,0
4,True,2014-04-01 16:00:02,25152,177.107.11.0/24,"[25152, 6939, 3549, 16735, 53059, 262730]",177.107.11.0,24,177.107.11.0/24,6,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20704,True,2014-04-01 16:59:55,25152,94.139.170.0/23,"[25152, 6939, 8529, 12880, 25124, 57218, 57218...",94.139.170.0,23,94.139.170.0/23,8,0,...,1,0,0,0,0,0,0,0,0,0
20705,True,2014-04-01 16:59:55,25152,188.253.120.0/22,"[25152, 6939, 8529, 48159, 58127]",188.253.120.0,22,188.253.120.0/22,5,0,...,0,0,0,0,0,0,0,0,0,0
20706,True,2014-04-01 16:59:56,25152,5.233.125.0/24,"[25152, 6939, 8529, 48159]",5.233.125.0,24,5.233.125.0/24,4,0,...,0,1,0,0,0,0,0,0,0,0
20707,True,2014-04-01 16:59:57,25152,66.210.60.0/24,"[25152, 6939, 22773, 20450]",66.210.60.0,24,66.210.60.0/24,4,0,...,0,1,0,0,0,0,0,0,0,0


In [41]:
del(predfspar)

In [52]:
# dfr.to_hdf("googletest-raw.h5", key="features", format="table")
dfr.to_json("../data/04_feature/googletest-features2.json")

In [43]:
# from pandas_profiling import ProfileReport

# profile = ProfileReport(dfr, minimal=True)
# # profile = ProfileReport(dfr[["type", 	"time" ,	"peer_asn" ,	"as_path" 	, 	"prefix_nm" ,	"prefix_ip_a" ,	"prefix_ip_b" ,	"prefix_ip_c", 	"prefix_ip_d", 	"prefix_ip_cat"]], minimal=True)
# profile.to_file("output.html")



Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

DispatchError: Function <code object pandas_get_series_descriptions at 0x7f8b92ff5e70, file "/home/cmos/.virtualenvs/bgpcompute-kedro/lib/python3.10/site-packages/pandas_profiling/model/pandas/summary_pandas.py", line 53>